In [ ]:
# 먼저 경로 변경
import os

os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [ ]:
# 일반담배와 전자담배흡연율 추이 불러오기
import pandas as pd

elect_data = pd.read_csv('액상형전자담배.csv', index_col=0).dropna().reset_index(drop=True)
nomal_data = pd.read_csv('일반담배.csv', index_col=0).dropna().reset_index(drop=True)
# 전체흡연율만 필요해서 슬라이싱함
nd = nomal_data.iloc[2][2:]
ed = elect_data.iloc[2][2:]

new_smoke = pd.concat([nd.astype(float),ed.astype(float)],axis=1) # 그래프 2개 합침
# 증감률 추이를 보여주기위해 diff, 그것을 누적으로 보여주기위해 cumsum사용
new_smoke = new_smoke.diff(axis=0).cumsum()

In [ ]:
# 일반 담배의 흡연율에 대한 추이 시각화
import matplotlib.pyplot as plt
import koreanize_matplotlib

nomal = pd.DataFrame(nd)
gr = nomal.astype(float).plot(kind='bar',figsize=[10,5]).get_legend().remove()
for i in range(len(nomal)):
  value = nomal.iloc[i,0]
plt.title('일반담배 흡연률 추이')
plt.xticks(rotation=0)
plt.yticks(range(0,31,5))
plt.show()

In [ ]:
# 평균 변화율 계산해보자

# 일단.. 두번째 행부터 float type으로 변경
sec_values = nomal_data.iloc[2, 3:].astype(float)

# 연도별 변화율 계산
change_rate = sec_values.pct_change()* 100    ## pct_change : 퍼센트 변화를 의미!

# 평균 변화율 계산
average_rate = change_rate.mean()

print(average_rate)

In [ ]:
# 확실한 증가 추이를 보이기 위해 2014, 2017, 2020, 2022년도 데이터만 좀 뽑아오자

selected_years = ['2014', '2017', '2020', '2022']
result = ed.loc[selected_years]

result

In [ ]:
# 추이에 대한 시각화

e_cigar = pd.DataFrame(result)
gr = e_cigar.astype(float).plot(kind='bar',figsize=[10,5]).get_legend().remove()
for i in range(len(nomal)):
  value = nomal.iloc[i,0]
plt.title('전자담배(액상형) 흡연율 증감 추이')
plt.xticks(rotation=0)
plt.yticks(range(0,8,2))
plt.show()

In [ ]:
# 마찬가지로 평균 변화율을 살펴보면,

# 일단.. 두번째 행부터 float type으로 변경
e_cigar_values = result.astype(float)

# 연도별 변화율 계산
change_rate_e = e_cigar_values.pct_change()* 100    ## pct_change : 퍼센트 변화를 의미!

# 평균 변화율 계산
average_rate_e = change_rate_e.mean()

print(average_rate)

In [ ]:
# 일단 연도를 맞추고,
selected_years_2 = ['2013', '2014', '2017', '2020', '2022']
nd_2 = nd.loc[selected_years_2]
ed_2 = ed.loc[selected_years_2]

# 새로운 변수를 통해 그래프 2개를 합침
new_smoke = pd.concat([nd_2.astype(float), ed_2.astype(float)], axis = 1) # 오 엑셀 concat 같다

# 증감률 추이를 보여주기 위해 diff, 그것을 누적으로 보여주기 위해 cumsum 사용
new_smoke = new_smoke.diff(axis = 0).cumsum()
new_smoke = new_smoke.loc["2014":]        ## 2013은 빈 값이 될 것이므로

# 시각화
new_smoke.plot(kind='bar', color=['skyblue', 'red'],figsize=[7,5])
plt.title('흡연율 증감')
plt.legend(['일반담배', '전자담배'])
plt.xticks(rotation=0)
plt.yticks(range(-6,6))
plt.show()

In [ ]:
# import 진행
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# selenium을 통한 동적 웹 크롤링으로 기사 정보 가지고 오기
driver = webdriver.Chrome()
driver.maximize_window()
news_info = []
for i in range(2,5):
    url = f'https://search.naver.com/search.naver?where=news&query=%ED%9D%A1%EC%97%B0&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=202{i}.01.01&de=202{i}.12.27&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20231227to20241227&is_sug_officeid=0&office_category=0&service_area=0'
    driver.get(url)
    time.sleep(2)

    for _ in range(120):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    news_list = driver.find_elements(By.CSS_SELECTOR,'#main_pack > section > div.api_subject_bx > div.group_news > ul > li')


    for news in news_list:

        try:
            press,link = news.find_elements(By.CSS_SELECTOR,'div.info_group > a')
            title= news.find_element(By.CSS_SELECTOR,'div.news_contents > a:nth-child(2)')
            print(press.text, title.text,link.get_attribute('href'))
            news_dict = {
                'year' : f"202{i}",
                'name' : press.text,
                'link' : link.get_attribute('href'),
                'title' : title.text
            }
            news_info.append(news_dict)
        except:
            pass


In [ ]:
# 판다스 활용, 크롤링 파일을 csv 파일로 변환
data = pd.DataFrame(news_info)

data.to_csv('네이버뉴스크롤링.csv')

In [ ]:
# 불러와서 확인
data = pd.read_csv("네이버뉴스크롤링.csv")

data.head()

In [ ]:
# 얼마나 검색되었을까?
data.value_counts("year")

In [ ]:
# 문장별 형태소 분석

import re
from konlpy.tag import Okt

okt = Okt()

word_list = []
stopwords = ["유아인", "대마", "대마초", "남양유업", "혐의", "손자", "마약", "프로포폴", "투약", "징역"]            ### 흡연과 관계 없는 단어 제외

i = 0
for review in data["title"].tolist():
    print(f"탐색할 리뷰: {review}")
    new_review = re.sub("[^0-9a-zA-Z가-힣\\s+]", "", review)
    new_review_pos = okt.pos(new_review, norm=True, stem=True)
    print(f"pos 결과: {new_review_pos}")
    # 한 문장에 대한 반복문
    print(f"review에서 조건 만족하는 단어들: ", end="")
    noun_list = []
    verb_list = []
    adj_list = []
    for word, pos in new_review_pos:
        if len(word) == 1 or word in stopwords:
            continue
        if pos == "Noun":
            print(word, end=" ")
            noun_list.append(word)
        elif pos == "Verb":
            print(word, end=" ")
            verb_list.append(word)
        elif pos == "Adjective":
            print(word, end=" ")
            adj_list.append(word)
    sub_dict = {"Noun": " ".join(noun_list), "Verb": " ".join(verb_list), "Adjective": " ".join(adj_list)}          ### 명사, 동사, 형용사로 쪼개어 분석

    word_list.append(sub_dict)
    print()
    print("="*100)

In [ ]:
# 위의 형태소 분석을 기반으로 일단 데이터 프레임 만들기

new_data = pd.concat([data, pd.DataFrame(word_list)], axis=1)
new_data.head()

In [ ]:
# 전체를 보기 위하여 'total'이라는 새 coulmns 추가

new_data["total"] = new_data["Noun"] + " " + new_data["Verb"] + " " + new_data["Adjective"]
new_data

In [ ]:
# 형태소 분석에 대해 따로 정리
new_data.to_csv('네이버뉴스크롤링_형태소분석.csv',index=False)

In [ ]:
# 내가 가지고 있는 폰트 목록 확인하기
from matplotlib import font_manager

search_font = "gothic"    # 필터

for font in font_manager.findSystemFonts():
  font_info = font_manager.FontProperties(fname=font)
  font_name = font_info.get_name()
  font_path = font_info.get_file()
  if search_font in font_name.lower():
    print(font_name, font_path)

In [ ]:
# 폰트 등록하기
from matplotlib import font_manager

## 폰트 경로
font_path = "/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf"
## 폰트 추가
font_manager.fontManager.addfont(font_path)
## 폰트 이름 확인
font_name = font_manager.FontProperties(fname=font_path).get_name()
print(font_name)

In [ ]:
import matplotlib.pyplot as plt     # 시각화를 위한 ㅎㅎ
# 폰트 설정하기
font_name = "NanumBarunGothic"
plt.rcParams["font.family"] = font_name

##### 본격 워드 클라우드

In [ ]:
# 연도별 워드 클라우드 그리기(명사), 22년 먼저
from collections import Counter
from wordcloud import WordCloud     # 워드 클라우드
import matplotlib.pyplot as plt     # 시각화

year = 2022

noun_list = []
for noun in new_data.loc[new_data["year"] == year, "Noun"]:
    noun_list.extend(noun.split(" "))
print(noun_list)

counter_22 = dict(Counter(noun_list).most_common(50))
wordcloud = WordCloud(
    font_path = "C:/Windows/Fonts/NanumBarunGothic.ttf"
).generate_from_frequencies(counter_22)

plt.imshow(wordcloud)
plt.title(f"{year} 워드클라우드")
plt.axis("off")
plt.show()

In [ ]:
# 연도별 워드 클라우드 그리기(명사) - 2023년
from collections import Counter
from wordcloud import WordCloud     # 워드 클라우드
import matplotlib.pyplot as plt     # 시각화

year = 2023

noun_list = []
for noun in new_data.loc[new_data["year"] == year, "Noun"]:
    noun_list.extend(noun.split(" "))
print(noun_list)

counter_23 = dict(Counter(noun_list).most_common(50))
wordcloud = WordCloud(
    font_path = "C:/Windows/Fonts/NanumBarunGothic.ttf"
).generate_from_frequencies(counter_22)

plt.imshow(wordcloud)
plt.title(f"{year} 워드클라우드")
plt.axis("off")
plt.show()

In [ ]:
# 연도별 워드 클라우드 그리기(명사) - 2024년
from collections import Counter
from wordcloud import WordCloud     # 워드 클라우드
import matplotlib.pyplot as plt     # 시각화

year = 2024

noun_list = []
for noun in new_data.loc[new_data["year"] == year, "Noun"]:
    noun_list.extend(noun.split(" "))
print(noun_list)

counter_24 = dict(Counter(noun_list).most_common(50))
wordcloud = WordCloud(
    font_path = "C:/Windows/Fonts/NanumBarunGothic.ttf"
).generate_from_frequencies(counter_22)

plt.imshow(wordcloud)
plt.title(f"{year} 워드클라우드")
plt.axis("off")
plt.show()

In [ ]:
# 위의 워드 클라우드에서 공통 명사를 추출

common_nouns = set(counter_22.keys()) & set(counter_23.keys()) & set(counter_24.keys())
common_nouns

In [ ]:
# 데이터 불러오기
data = pd.read_csv("네이버뉴스크롤링_전자담배_형태소분석.csv")

data

In [ ]:
# 연도별 워드 클라우드 그리기(전체) - 2022년

year = 2022

noun_list = []
for noun in data.loc[data["year"] == year, "total"]:
    noun_list.extend(noun.split(" "))
print(noun_list)

counter_22 = dict(Counter(noun_list).most_common(50))
wordcloud = WordCloud(
    font_path = "C:/Windows/Fonts/NanumBarunGothic.ttf"
).generate_from_frequencies(counter_22)

plt.imshow(wordcloud)
plt.title(f"{year} 워드클라우드")
plt.axis("off")
plt.show()

In [ ]:
# 연도별 워드 클라우드 그리기(전체) - 2023년

year = 2023

noun_list = []
for noun in data.loc[data["year"] == year, "total"]:
    if isinstance(noun, str):
      noun_list.extend(noun.split())
print(noun_list)

counter_23 = dict(Counter(noun_list).most_common(50))
wordcloud = WordCloud(
    font_path = "C:/Windows/Fonts/NanumBarunGothic.ttf"
).generate_from_frequencies(counter_23)

plt.imshow(wordcloud)
plt.title(f"{year} 워드클라우드")
plt.axis("off")
plt.show()

In [ ]:
# 연도별 워드 클라우드 그리기(전체)

year = 2024

noun_list = []
for noun in data.loc[data["year"] == year, "total"]:
    if isinstance(noun, str):
      noun_list.extend(noun.split())
print(noun_list)

counter_24 = dict(Counter(noun_list).most_common(50))
wordcloud = WordCloud(
    font_path = "C:/Windows/Fonts/NanumBarunGothic.ttf"
).generate_from_frequencies(counter_24)

plt.imshow(wordcloud)
plt.title(f"{year} 워드클라우드")
plt.axis("off")
plt.show()

In [ ]:
# 마찬가지로 공통 단어 뽑아보기

common_nouns = set(counter_22.keys()) & set(counter_23.keys()) & set(counter_24.keys())
common_nouns

In [ ]:
from apyori import apriori
import itertools
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
# 데이터 불러오기

wc = pd.read_csv('keyword_전자담배.csv', index_col=0).dropna().reset_index(drop=True)
okt = Okt()
# desc = wc['description']
title = wc['title']
stopwords = ['대구시', '상형', '하중', '의원']          ### 쓸모없는 단어 방출 ㅎ

def divied(array):
    word_list = []
    for review in array.tolist():
        edit_review = re.sub('[^0-9a-zA-Z가-힣\\s+-]', '', review)
        edit_review_pos = okt.pos(edit_review)
        sub_list = []

        for word, pos in edit_review_pos:
            if len(word) < 2 or word in stopwords:
                continue
            elif pos =="Noun":
                sub_list.append(word)
        word_list.append(sub_list)
    return word_list

In [ ]:
nsize = None

def ori(array):
    # 연관된 단어들을 추출
    result = list(apriori(divied(array)))
    new_data = pd.DataFrame(result)
    return new_data.sort_values(by='support', ascending=False)

In [ ]:
def net(array):
    data = ori(array)  # 문제발생
    lengths = []

    for item in data['items'].tolist():  # item 항목이 2개 미만을 거르는 작업
        lengths.append(len(item))
    data['length'] = lengths
    new_data2 = data[data['length']>1]

    plt.rc('font', family = 'NanumBarunGothic')

    G = nx.Graph()
    word_combi = []
    for item in new_data2['items'].tolist(): #연관된 단어들을 2개씩 조합하여 리스트에 저장
        word_combi.extend(itertools.combinations(item, 2))
    G.add_edges_from(word_combi) # 네트워크 그래프에 간선을 추가

    pr = nx.pagerank(G) # pagerank를 사용하여 노드의 중요도를 계산하여 딕셔너리 형태로 저장
    nsize = np.array(list(pr.values()))
    nsize = 2000* (nsize - min(nsize)) / (max(nsize) - min(nsize)) # 노드 크기

    lay = nx.kamada_kawai_layout(G) # 노드간 간선을 시각적으로 균형있게 배치치
    nx.draw(G, lay, node_size = nsize, with_labels=True, font_size = 10, font_family = 'NanumGothic')
    plt.title('담배 네트워크')
    plt.show

    return None

print(net(title))

In [ ]:
# 연도별 구매루트 비율
route = pd.read_csv('전자담배구매루트.csv',index_col=0).dropna().reset_index(drop=True)
route.iloc[:,3][2]

# 루트별 구입비중 구하기기
df = []
for i in range(len(route.columns)):
    if route.iloc[:,i][2] == "구입한 적 있음":
        df.append(route.iloc[:,i][0:4:3])
df = pd.DataFrame(df)


market = df.iloc[:,1][0::4].astype(float)
store = df.iloc[:,1][1::4].astype(float)
pc = df.iloc[:,1][2::4].astype(float)
internet = df.iloc[:,1][3::4].astype(float)

import numpy as np

arr = np.arange(2016,2021,2) # y축값 저장장
w = 0.3
plt.bar(arr-w,market,width=0.3, alpha=0.5, color='blue', label='편의점')
plt.bar(arr,store,width=0.3, alpha=0.5, color='orange', label='전자담배판매점')
plt.bar(arr+w,pc,width=0.3, alpha=0.5, color='green',label='pc방 노래방')
plt.bar(arr+w+w,internet,width=0.3, alpha=0.5, color='red', label='인터넷')
plt.xticks(range(2016,2021,2))
# plt.Subplots(figsize=[12,6])
plt.legend()
plt.title('전자담배 구매루트 추이')

In [ ]:
import numpy as np
import pandas as pd

data = pd.DataFrame({
    "온라인 쇼핑몰": ["네이버", "지마켓", "11번가", "옥션"],        ## 인터넷 쇼핑 대표 도메인으로 설정
    "검색 결과": [69833, 19745, 12687, 10432]                       ## 실제 검색되는 수를 바탕으로 만듦
})
data

In [ ]:
# 위의 데이터프레임을 바탕으로 시각화 진행

fig, ax = plt.subplots(figsize=(8, 5))  # Axes 객체 생성
bars = ax.bar(data["온라인 쇼핑몰"], data["검색 결과"], zorder=2,color=['red','blue', 'green', 'yellow'])

#plt.figure(figsize=(8, 5))  # 차트 크기 설정
#plt.bar(data["온라인 쇼핑몰"], data["검색 결과"], zorder=2)

#격자 만들기
plt.grid(axis='y', linestyle='--', alpha=0.5, zorder=1)

# 라벨 달아주기
plt.title("온라인 쇼핑 플랫폼 별 검색 결과 건수", fontsize = 16)
plt.xlabel("온라인 쇼핑 플랫폼", fontsize = 12)
plt.ylabel("'전자담배' 검색 시 결과 수", fontsize = 12)

# Y축 값 표시
ax.bar_label(bars, labels=[f'{v:,}' for v in data["검색 결과"]], fontsize=10, padding=3)
plt.yticks(range(0,100001,10000))
plt.tight_layout()
plt.show()

#최종 업데이트 : 자료 기준 24.12.27 플랫폼 검색 기준 (중고, 해외, 렌탈 제외 )